In [1]:
library(dplyr)
library(MASS)
library(iMKT)
root_dir <- '~/Escritorio/mastersthesis/'
data_dir <- paste(root_dir,'data/', sep='')
scripts_dir <- paste(root_dir, 'scripts/', sep='')
results_dir <- paste(root_dir, 'results/', sep='')



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: ggplot2



In [2]:
#' @title iMKT using PopHuman data
#'
#' @description Perform any MKT method using a subset of PopHuman data defined by custom genes and populations lists
#'
#' @details Execute any MKT method (standardMKT, FWW, eMKT, aMKT) using a subset of PopHuman data defined by custom genes and populations lists. It uses the dataframe PopHumanData, which can be already loaded in the workspace (using loadPopHuman()) or is directly loaded when executing this function. It also allows deciding whether to analyze genes groupped by recombination bins or not, using recombination rate values corresponding to the sex average estimates from Bhérer et al. 2017 Nature Commun. 
#'
#' @param genes list of genes to analyze
#' @param pops list of populations to analyze
#' @param cutoffs list of cutofs to perform FWW and/or eMKT
#' @param test which test to perform. Options include: standardMKT (default), eMKT, FWW, aMKT
#' @param xlow lower limit for asymptotic alpha fit (default=0)
#' @param xhigh higher limit for asymptotic alpha fit (default=1)
#' @param plot report plot (optional). Default is FALSE
#' 
#' @return List of lists with the default test output for each selected population (and recombination bin when defined)
#'
#' @examples
#' ## List of genes
#' mygenes <- c("ENSG00000011021.21_3","ENSG00000091483.6_3","ENSG00000116191.17_3",
#'							"ENSG00000116337.15_4","ENSG00000116584.17_3","ENSG00000116745.6_3",
#'							"ENSG00000116852.14_3","ENSG00000116898.11_3","ENSG00000117010.15_3",
#'							"ENSG00000117090.14_3","ENSG00000117222.13_3","ENSG00000117394.20_3")
#' ## Perform analyses
#' PopHumanAnalysis(genes=mygenes , pops=c("CEU","YRI"), test="standardMKT")
#' PopHumanAnalysis(genes=mygenes , pops=c("CEU"), test="eMKT")
#' 
#' @import utils
#' @import stats
#'
#' @keywords PopData
#' @export
			
PopHumanAnalysisMod <- function(genes=c("gene1","gene2","..."), pops=c("pop1","pop2","..."), cutoff=0.05, test=c("standardMKT","eMKT","FWW","aMKT"), xlow=0, xhigh=1, plot=FALSE, cum=FALSE, daf10=FALSE) { 
	
	## Get PopHuman data
	if (exists("PopHumanData") == TRUE) {
	data <- get("PopHumanData")
	} else {
	loadPopHuman()
	data <- get("PopHumanData") }
	
	## Check input variables
	## Numer of arguments
	if (nargs() < 2 && nargs()) {
	stop("You must specify 2 arguments at least: genes, pops.\nIf test = asymptoticMKT or test = aMKT, you must specify xlow and xhigh values.") }
	
	## Argument genes
	if (length(genes) == 0 || genes == "" || !is.character(genes)) {
	stop("You must specify at least one gene.") }
	if (!all(genes %in% data$ID) == TRUE) {
	difGenes <- setdiff(genes, data$ID)
	difGenes <- paste(difGenes, collapse=", ")
	stopMssg <- paste0("MKT data is not available for the requested gene(s).\nRemember to use gene IDs from Ensembl (ENSG...).\nThe genes that caused the error are: ", difGenes, ".")
	stop(stopMssg) }
	
	## Argument pops
	if (length(pops) == 0 || pops == "" || !is.character(pops)) {
	stop("You must specify at least one population.") }
	if (!all(pops %in% data$Population) == TRUE) {
	correctPops <- unique(data$Population)
	difPops <- setdiff(pops, correctPops)
	difPops <- paste(difPops, collapse=", ")
	stopMssg <- paste0("MKT data is not available for the sequested populations(s).\nSelect among the following populations:\n",correctPops,"\nThe populations that caused the error are: ", difPops, ".")
	stop(stopMssg) }
	
	## Argument test and xlow + xhigh (when necessary)
	if(missing(test)) {
	test <- "standardMKT"
	}
	else if (test != "standardMKT" && test != "eMKT" && test != "FWW" && test != "aMKT") {
	stop("Parameter test must be one of the following: standardMKT, eMKT, FWW, aMKT, iMKT")
	}
	if (length(test) > 1) {
	stop("Select only one of the following tests to perform: standardMKT, eMKT, FWW, aMKT") }
	if ((test == "standardMKT" || test == "eMKT" || test == "FWW") && (xlow != 0 || xhigh != 1)) {
	warningMssgTest <- paste0("Parameters xlow and xhigh not used! (test = ",test," selected)")
	warning(warningMssgTest) }
	
	## Arguments xlow, xhigh features (numeric, bounds...) checked in checkInput()
	
	## Perform subset
	subsetGenes <- data[(data$ID %in% genes & data$Population %in% pops), ]
	subsetGenes$ID <- as.factor(subsetGenes$ID)
	subsetGenes <- droplevels(subsetGenes)
	
	## Declare output list (each element 1 pop)
	outputList <- list()
	
	for (i in levels(subsetGenes$Population)) {
		# print(paste0("Population = ", i))
		x <- subsetGenes[subsetGenes$Population == i, ]
		
		## Set counters to 0
		Pi <- c(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
		P0 <- c(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
		f <- seq(0.025,0.975,0.05)
		mi <- 0; m0 <- 0
		Di <- 0; D0 <- 0
		
		## Group genes
		for (j in levels(x$ID)) {
		x1 <- x[x$ID == j, ]
		
		## DAF
		x1$DAF0f <- as.character(x1$DAF0f); x1$DAF4f <- as.character(x1$DAF4f)
		daf0f <- unlist(strsplit(x1$DAF0f, split=";"))
		daf4f <- unlist(strsplit(x1$DAF4f, split=";"))
		daf0f <- as.numeric(daf0f); daf4f <- as.numeric(daf4f)
		Pi <- Pi + daf0f; P0 <- P0 + daf4f
		
		## Divergence
		mi <- mi + x1$mi; m0 <- m0 + x1$m0
		Di <- Di + x1$di; D0 <- D0 + x1$d0
		}

		## Proper formats
		daf <- cbind(f, Pi, P0); daf <- as.data.frame(daf)
		names(daf) <- c("daf","Pi","P0")
		div <- cbind(mi, Di, m0, D0); div <- as.data.frame(div)
		names(div) <- c("mi","Di","m0","D0")
		

		if (isTRUE(cum)){
      daf <- cumulative(daf)
		}
		

		## Check data inside each test!
		## Queremos que vaya de 0.05 en 0.05 si los genes son suficientes.
		# Transform daf20 to daf10 (faster fitting) for asymptoticMKT and aMKT
  	if (isTRUE(daf10)){
			if (nrow(daf) == 20) {
  		daf1 <- daf
  		daf1$daf10 <- sort(rep(seq(0.05,0.95,0.1),2)) ## Add column with the daf10 frequencies
        print(daf1$daf10)
  		daf1 <- daf1[c("daf10","Pi","P0")] ## Keep new frequencies, Pi and P0
  		daf1 <- aggregate(. ~ daf10, data=daf1, FUN=sum)	## Sum Pi and P0 two by two based on daf
  		colnames(daf1)<-c("daf","Pi","P0") ## Final daf columns name
  		
  		daf <- daf1
  		}
  	}

		## Perform test
		if(test == "standardMKT") {
		output <- standardMKT(daf, div) }
		else if(test == "eMKT" && plot == FALSE) {
		output <- eMKT(daf, div,listCutoffs=cutoff) }
		else if(test == "eMKT" && plot == TRUE) {
		output <- eMKT(daf, div,listCutoffs=cutoff, plot=TRUE) }
		else if(test == "FWW" && plot == FALSE) {
		output <- FWW(daf, div, listCutoffs=cutoff) }
		else if(test == "FWW" && plot == TRUE) {
		output <- FWW(daf, div, listCutoffs=cutoff, plot=TRUE) }
		else if(test == "aMKT" && plot == FALSE) {
		output <- aMKT(daf, div, xlow, xhigh) }
		else if(test == "aMKT" && plot == TRUE) {
		output <- aMKT(daf, div, xlow, xhigh, plot=TRUE) }
		
		## Fill list with each pop
		outputList[[paste("Population = ",i)]] <- output
	}
	
	## Return output
	cat("\n")
	return(outputList)
	
}


In [3]:
cumulative <- function(x){

	psyn  = rep(0,length(x$daf))
	pnsyn = rep(0,length(x$daf))
	psyn[1]  = sum(x$P0)
	pnsyn[1] = sum(x$Pi)

	for(i in 2:length(x$daf)){
		appS    = psyn[i-1] - x$P0[i-1]
		appNsyn = pnsyn[i-1] - x$Pi[i-1]

		if( appS > 0.0 & appNsyn > 0.0){
			psyn[i]  = appS
			pnsyn[i] = appNsyn
		}
		else{
			psyn[i]  = 0
			pnsyn[i] = 0		
		}

	}

	x$Pi = pnsyn 
	x$P0 = psyn

	return(x)
}

In [4]:
makeSfs <- function(x,cumu=FALSE){

	f <- seq(0.025,0.975,0.05)
	pi <- do.call(rbind,lapply(as.character(x[['DAF0f']]), function(z) unlist(strsplit(z, split = ';')) %>% as.numeric())) %>% colSums()
	p0 <- do.call(rbind,lapply(as.character(x[['DAF4f']]), function(z) unlist(strsplit(z, split = ';')) %>% as.numeric())) %>% colSums()
	mi <- x[['mi']]
	m0 <- x[['m0']]
	di <- x[['di']]
	d0 <- x[['d0']]
    
	## Proper formats
	daf <- cbind(f, pi, p0) %>% as.data.frame()
	names(daf) <- c('daf','Pi','P0')
	div <- cbind(mi, di, m0, d0) %>% colSums %>% t %>%as.data.frame() 
	names(div) <- c('mi','Di','m0','D0')

	if(cumu == TRUE){

		out<-list()
		out[['daf']] <- cumulative(daf)
		out[['div']] <- div
		return(out)

	}else{

		out<-list()
		out[['daf']] <- daf
		out[['div']] <- div
		return(out)

	}
}

In [5]:
#' @title eMKT correction method
#'
#' @description MKT calculation corrected using eMKT method (Mackay et al. 2012 Nature).
#'
#' @details In the standard McDonald and Kreitman test, the estimate of adaptive evolution (alpha) can be easily biased by the segregation of slightly deleterious non-synonymous substitutions. Specifically, slightly deleterious mutations contribute more to polymorphism than they do to Divergence, and thus, lead to an underestimation of alpha. Because adaptive mutations and weakly deleterious selection act in opposite Directions on the MKT, alpha and the fraction of substitutions that are slighlty deleterious, b, will be both underestimated when both selection regimes occur. To take adaptive and slighlty deleterious mutations mutually into account, Pi, the count off segregatning sites in class i, should be separated into the number of neutral variants and the number of weakly deleterious variants, Pi = Pineutral + Pi weak del. Alpha is then estimated as 1-(Pineutral/P0)(D0/Di). As weakly deleterious mutations tend to segregate at low frequencies, neutral and weakly deleterious fractions from Pi can be estimated based on any frequency cutoff established.
#'
#' @param daf data frame containing DAF, Pi and P0 values
#' @param Divergence data frame containing Divergent and analyzed sites for selected (i) and neutral (0) classes
#' @param listCutoffs list of cutoffs to use (optional). Default cutoffs are: 0, 0.05, 0.1
#' @param plot report plot (optional). Default is FALSE
#' 
#' @return MKT corrected by the eMKT method. List with alpha results, graph (optional), Divergence metrics, MKT tables and negative selection fractions
#'
#' @examples
#' ## Using default cutoffs
#' eMKT(myDafData, myDivergenceData)
#' ## Using custom cutoffs and rendering plot
#' eMKT(myDafData, myDivergenceData, c(0.05, 0.1, 0.15), plot=TRUE)
#'
#' @import utils
#' @import stats
#' @import ggplot2
#' @importFrom ggthemes theme_foundation
#' @importFrom cowplot plot_grid
#' @importFrom reshape2 melt 
#'
#' @keywords MKT
#' @export


####################################################
################# MKT-FWW function #################
####################################################

eMKT = function(daf, divergence, listCutoffs=0.15, plot=FALSE) {
	
	## Check data
# 	check = checkInput(daf, divergence, 0, 1)
# 	if(check$data == FALSE)
# 	{
# 		 stop(check$print_errors) 
# 	}
	
	## Declare output lists and data frames
	output     = list()
	mktTables  = list()
	divMetrics = list()
	divCutoff  = list()
	
	##Polymorphism and Divergence variables
	P0 = sum(daf[['P0']])
	Pi = sum(daf[['Pi']])
	D0 = divergence[['D0']]
	Di = divergence[['Di']]
	m0 = divergence[['m0']]
	mi = divergence[['mi']]
	
	## MKT tables
	mktTableStandard = data.frame(Polymorphism = c(sum(daf[['P0']]), sum(daf[['Pi']])), Divergence = c(D0,Di),row.names = c("Neutral class","Selected class"))

    ## Divergence metrics
	Ka              = Di/mi
	Ks              = D0/m0
	omega           = Ka/Ks

	## Estimation of alpha
	# alpha = 1 - ((Pi/P0)*(D0/Di))
	# alphaC = 1 - ((PiNeutral/P0)*(mktTableStandard[1,2]/mktTableStandard[2,2]))
	alphaCorrected <- list()
	fractions <- list()
	for (c in listCutoffs) {
		
		## Estimating alpha with Pi/P0 ratio 
		PiMinus     = sum(daf[daf[['daf']] <= c,'Pi'])
		PiGreater   = sum(daf[daf[['daf']] > c,'Pi'])
		P0Minus     = sum(daf[daf[['daf']] <= c,'P0'])
		P0Greater   = sum(daf[daf[['daf']] > c,'P0'])
		ratioP0     = P0Minus/P0Greater
		deleterious = PiMinus - (PiGreater * ratioP0)
		PiNeutral = Pi - deleterious

		alphaC = 1 - (((Pi - deleterious)/P0)*(D0/Di))


		## Estimation of b: weakly deleterious
		b    = (deleterious/P0)*(m0/mi)
		
		## Estimation of f: neutral sites
		f = (m0*PiNeutral)/(as.numeric(mi)*as.numeric(P0))
		
		## Estimation of d, strongly deleterious sites
		d = 1 - (f+b)
		
		## Fisher exact test p-value from the MKT
		m      = matrix(c(P0,(Pi - deleterious),D0,Di), ncol=2)
		pvalue = fisher.test(round(m))$p.value
		
		## Omega A and Omega D
		omegaA = omega * alphaC
		omegaD = omega - omegaA
		
		## Store output  
		alphaCorrected[[paste0('cutoff=',c)]] = c(c, alphaC, pvalue)
		fractions[[paste0('cutoff=',c)]] = c(c, d, f, b)
		divCutoff[[paste0('cutoff=',c)]] = c(c, Ka,Ks,omegaA, omegaD)
		# mktTables[[paste0('cutoff=',c)]]  = mktTableCleaned
	}

	## Fractions
	# names(fraction) = 'Correction'
	
	## Store output 
	## Output format
	output[['mktTable']]                 = mktTableStandard 
	output[['alphaCorrected']]           = as.data.frame(do.call('rbind',alphaCorrected))
	colnames(output[['alphaCorrected']]) = c('cutoff', 'alphaCorrected', 'pvalue')
	## Divergence metricss
	divCutoff                            = as.data.frame(do.call('rbind',divCutoff))
	names(divCutoff)                     = c('cutoff', 'Ka','Ks','omegaA', 'omegaD')
	output[['divMetrics']]               = list('metricsByCutoff'=divCutoff)
	output[['fractions']]                = as.data.frame(do.call('rbind',fractions))
	names(output[['fractions']])                     = c('cutoff', 'd','f','b')
	
	# DivCutoff                     = data.frame('omegaA' = omegaA, 'omegaD' = omegaD)
	# output[['divMetrics']]        = list(DivTable, DivCutoff)
	# names(output[['divMetrics']]) = c("Global metrics", "Estimates by cutoff")
	# Results table
	# output = as.data.frame(do.call("rbind",output))
	# colnames(output) = c("cutoff", "alpha", "pvalue")
	
	# Divergence metrics
	# DivCutoff = as.data.frame(do.call("rbind",DivCutoff))
	# names(DivCutoff) = c("omegaA", "omegaD")

	## Render plot
	if (plot == TRUE) {

		## Cut-offs graph
		# plot = ggplot(output, aes(x=as.factor(cutoff), y=alpha, group=1)) +
		# 	geom_line(color="#386cb0") + 
		# 	geom_point(size=2.5, color="#386cb0")+
		# 	themePublication() +
		# 	xlab("Cut-off") + ylab(expression(bold(paste("Adaptation (",alpha,")")))) 
	
		## Re-format outputs
		# output = output[,c(2,3)]
		# names(output) = c("alpha.symbol","Fishers exact test P-value")
		# DivCutoff = DivCutoff[,c(2,3)]
		# colnames(DivCutoff) = c("omegaA.symbol", "omegaD.symbol")
		# DivMetrics = list(DivTable, DivCutoff)
		# names(DivMetrics) = c("Global metrics", "Estimates by cutoff")
	
		## Melt fractions data
		plotAlpha = ggplot(output[['alphaCorrected']], aes(x=as.factor(cutoff), y=alphaCorrected, group=1)) +
			geom_line(color="#386cb0") + 
			geom_point(size=2.5, color="#386cb0")+
			themePublication() +
			xlab("Cut-off") + ylab(expression(bold(paste("Adaptation (",alpha,")"))))
	
		## Fractions graph
		i = which.max(output$alphaCorrected$alphaCorrected)
		fractionsMelt = output[['fractions']][i,2:4]
		fractionsMelt = reshape2::melt(fractionsMelt, id.vars=NULL) 
		fractionsMelt[['test']] = rep(c('eMKT'),3)

		plotFraction = ggplot(fractionsMelt) + geom_bar(stat="identity", aes_string(x="test", y="value", fill="variable"), color="black") +
			coord_flip() + themePublication() + ylab(label="Fraction") + xlab(label="Cut-off") +
			scale_fill_manual(values=c("#386cb0","#fdb462","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33"), breaks=c("f","d","b"), labels=c(expression(italic("f")),expression(italic("d")),expression(italic("b")))) +
			theme(axis.line=element_blank()) + scale_y_discrete(limit=seq(0,1,0.25), expand=c(0,0))
	
		plotEmkt = plot_grid(plotAlpha, plotFraction, nrow=2,  labels=c('A','B'), rel_heights=c(2,1))

		output[['graph']] = plotEmkt

		# plot = plot_grid(plot, plotfraction, nrow=2, labels=c("A","B"), rel_heights=c(2,1))
	
		## Return list output  
		# output[['Graph']] = plot
		# names(listOutput) = c("Results","Graph", "Divergence metrics", "MKT tables","Fractions")

		return(output)

	## If no plot to render
	} else if (plot==FALSE) {
			## Re-format outputs
			# output = output[,c(2,3)]
			# names(output) = c("alpha.symbol","Fishers exact test P-value")
			# DivCutoff = DivCutoff[,c(2,3)]
			# colnames(DivCutoff) = c("omegaA.symbol", "omegaD.symbol")
			# DivMetrics = list(DivTable, DivCutoff)
			# names(DivMetrics) = c("Global metrics", "Estimates by cutoff")
		
			# ## Melt fractions data
			# fractionsMelt = melt(fractions, id.vars=NULL) 
			# fractionsMelt$Fraction =  rep(c("d", "f", "b"),length(fractionsMelt$variable)/3)
			
			# ## Return list output  
			# listOutput = list(output, DivMetrics, mktTables, fractions)
			# names(listOutput) = c("Results", "Divergence metrics", "MKT tables","Fractions")

			return(output)
	}
	
}



# for (f in 2:nrow(daf)-1) {
# 	cleanedDafBellow = cleanedDaf[1, ]
# 	cleanedDafAbove  = cleanedDaf[2:nrow(cleanedDaf), ] ## over cleanedDaf 
	
# 	## Create MKT table 
# 	mktTable  = data.frame(`cleanedDaf below cutoff` = c(sum(cleanedDafBellow$P0), sum(cleanedDafBellow$Pi)), `cleanedDaf above cutoff`=c(sum(cleanedDafAbove$P0), sum(cleanedDafAbove$Pi)),row.names = c("Neutral class","Selected class"))
	
# 	cleanedPi = sum(cleanedDaf$Pi)
	
# 	## Estimate fractions
# 	fNeutral  = mktTable[1,1]/sum(cleanedDaf$P0)
# 	PiNeutralBelowCutoff = cleanedPi * fNeutral

# 	# If we haven't deleterious in Pi then PiNeutral = Pi_in_current_cutoff
# 	if(PiNeutralBelowCutoff > cleanedDafBellow[['Pi']]){
# 		PiNeutral = PiNeutral + cleanedDafBellow[['Pi']]
# 	}
# 	else{
# 		PiNeutral = PiNeutral + PiNeutralBelowCutoff
# 	}
	
# 	## Deleting current frequency 
# 	cleanedDaf = cleanedDaf[2:nrow(cleanedDaf),]

# } 

In [6]:
aMKT = function(daf, divergence, xlow=0, xhigh=1) {

	## Create MKT table standard
	mktTableStandard = data.frame(
		Polymorphism = c(sum(daf[['P0']]), sum(daf[['Pi']])), 
		Divergence   = c(divergence[['D0']],divergence[['Di']]),
		row.names    = c('Neutral class','Selected class')
	)
	
	##Polymorphism and Divergence variables
	P0 = sum(daf[['P0']])
	Pi = sum(daf[['Pi']])
	D0 = divergence[['D0']]
	Di = divergence[['Di']]
	m0 = divergence[['m0']]
	mi = divergence[['mi']]


	## Run asymptotic MKT and retrieve alphas 
	## iMKT only fits exponential model in asymptotic alpha. it uses asymptoticMKExp()

	asymptoticMkTable = asymptoticMKExp(daf, divergence, xlow, xhigh)
	alphaAsymptotic   = as.numeric(asymptoticMkTable[['alphaAsymptotic']])
	alphaStandard     = as.numeric(asymptoticMkTable[['alphaOriginal']])
	alphaCiLow        = asymptoticMkTable[['ciLow']] 
	alphaCiHigh       = asymptoticMkTable[['ciHigh']] 
		
	## Estimate alpha for each DAF category
	daf[['alpha']] = 1-((as.numeric(D0)*as.numeric(daf[['Pi']]))/(as.numeric(Di)*as.numeric(daf[['P0']])))

    ## Estimate the synonymous and non-synonymous ratio
	synonymousRatio    = P0/m0
	nonSynonymousRatio = Pi/mi 
	
	## Estimate the fraction of neutral sites incluiding weakly deleterious variants
	fb  = nonSynonymousRatio/synonymousRatio
	
	## Estimate the fraction of strongly deleleterious sites (d)
	d  = 1 - fb
	
	## Estimate the fraction of sligthly deleterious sites in each daf category (b)
	wd = daf[['Pi']] - (((1 - alphaAsymptotic) * divergence[['Di']] *  daf[['P0']])/divergence[['D0']])
	print(wd)
    b  = (sum(wd)/sum(daf[['P0']]))*(m0/mi)

	## Re-estimate the truly number of neutral sites, removing the slightly deleterious 
	f = fb - b

# 	fraction = data.frame(d = d, f = f, b = b)

	## Return output
	return(asymptoticMkTable) 
}

In [7]:
# Asymptotic execution
asymptoticMKExp = function(daf, divergence, xlow, xhigh, seed=NULL) {
	
	## Check data
	# check = checkInput(daf, divergence, xlow, xhigh)
	
	# if(check$data == FALSE) {
	# 	stop(check$print_errors) }
	
	if (any(daf$P0 == 0)){ ## Warning P0
		warning('Input daf file contains P0 values = 0.\nThis can bias the function fitting and the estimation of alpha.')}
	
# 	## Check seed
# 	if(missing(seed)) {
# 		seed = NULL
# 	} else {
# 		set.seed(seed)
# 	}
	
	## Parse the data from argument x
	f  = daf$daf #derived alelle frequencies
	p  = daf$Pi #non-synonymous polymorphism 
	p0 = daf$P0 #synonymous polymorphism
# 	print(daf)
	## Parse the data from argument y
	m  = divergence$mi #number of non-synonymous analyzed positions   
	m0 = divergence$m0 ##number of synonymous analyzed positions
	d  = divergence$Di #non-synonymous divergence
	d0 = divergence$D0 #synonymous divergence
	
	## Compute alpha values and trim
#     print(d0/d)
#     print(p/p0)
#     print(as.numeric(d0/d) * (p/p0))
	alpha         = 1 - (d0/d) * (p/p0)
	cutoff_f1     = xlow
	cutoff_f2     = xhigh
	trim          = ((f >= cutoff_f1) & (f <= cutoff_f2))
	f_trimmed     = f[trim]
	alpha_trimmed = alpha[trim]
# 	print('alpha')
#     print(alpha)
#     print('cutoff_f1')
#     print(cutoff_f1)
#     print('cutoff_f2')
#     print(cutoff_f2)
#     print('trim')
#     print(trim)
#     print('f_trimmed')
#     print(f_trimmed)
#     print('alpha_trimmed')
#     print(alpha_trimmed)
	## Compute the original MK alpha
	alpha_nonasymp = 1 - (d0/d) * (sum(p[trim])/sum(p0[trim])) #using trimmed values
	## Two-step nls2() model fit at a given level of precision (res)
    mod1 = fitMKmodel(alpha_trimmed, f_trimmed, 10)

    ## If mod1 did not work, try a deeper scan for a decent fit (res=20)
	if (length(mod1) == 0) {
		mod1 = fitMKmodel(alpha_trimmed, f_trimmed, 20)
	} 
	tryCatch({
		mod2 = lm(alpha_trimmed ~ f_trimmed)
	}, error=function(cond) {})
	
	tryCatch({
		ci_pred = predictNLS(mod1, newdata=data.frame(f_trimmed=1.0))
		alpha_1_low = ci_pred[6]
		alpha_1_high = ci_pred[7]

		## Preparation of ouput (alpha asym, a, b, c)
		alpha_1_est = predict(mod1, newdata=data.frame(f_trimmed=1.0))
		const_a = coef(mod1)['const_a']
		const_b = coef(mod1)['const_b']
		const_c = coef(mod1)['const_c']
		
		## Output table
		result_df = data.frame(model='exponential', a=const_a, b=const_b, c=const_c, alphaAsymptotic=alpha_1_est, ciLow=alpha_1_low, ciHigh=alpha_1_high, alphaOriginal=alpha_nonasymp, row.names=NULL)
		return(result_df)
	}, error=function(cond) {cat('Could not fit exponential model for the computation of asymptotic alpha.\n')})
}

In [8]:
## Compute confidence intervals of alpha using predictNLS 
## Get a CI using Monte Carlo simulation based upon a fitted model.  
## Thanks to Andrej-Nikolai Spiess (http://www.dr-spiess.de) for this code.
predictNLS = function(object, newdata, level = 0.95, nsim = 10000) {
	  
	## get right-hand side of formula
	RHS  = as.list(object$call$formula)[[3]]
#     print('RHS')
#     print(RHS)
	EXPR = as.expression(RHS)
#     print('EXPR')
#     print(EXPR)
	
	## all variables in model
	VARS = all.vars(EXPR)
#     print('VARS')
#     print(VARS)
	
	## coefficients
	COEF = coef(object)
#     print('COEF')
#     print(COEF)
	
	## extract predictor variable    
	predNAME = setdiff(VARS, names(COEF))  
#     print(predNAME)
	
	## take fitted values, if 'newdata' is missing
	if (missing(newdata)) {
		newdata           = eval(object$data)[predNAME]
		colnames(newdata) = predNAME
	}
	  
	## check that 'newdata' has same name as predVAR
	if (names(newdata)[1] != predNAME) stop("newdata should have name '", predNAME, "'!")

	## get variance-covariance matrix
	VCOV = vcov(object)
#     print('VCOV')
#     print(VCOV)

	## augment variance-covariance matrix for 'mvrnorm' 
	## by adding a column/row for 'error in x'
	NCOL = ncol(VCOV)
	ADD1 = c(rep(0, NCOL))
	ADD1 = matrix(ADD1, ncol = 1)
	colnames(ADD1) = predNAME
	VCOV = cbind(VCOV, ADD1)
	ADD2 = c(rep(0, NCOL + 1))
	ADD2 = matrix(ADD2, nrow = 1)
	rownames(ADD2) = predNAME
	VCOV = rbind(VCOV, ADD2) 

	## iterate over all entries in 'newdata' as in usual 'predict.' functions
	NR = nrow(newdata)
	respVEC = numeric(NR)
	seVEC = numeric(NR)
	varPLACE = ncol(VCOV)   

	## define counter function
	counter = function(i) {
		if (i%%10 == 0) { cat(i) 
	} else { cat(".") }
		if (i%%50 == 0) { cat("\n") }
		flush.console()
	}
	  
	## create output matrix (df)
	outMAT = NULL 
	
	for (i in 1:NR) {
		
		## get predictor values and optional errors
		predVAL = newdata[i, 1]
		if (ncol(newdata) == 2) predERROR = newdata[i, 2] else predERROR = 0
		names(predVAL) = predNAME  
		names(predERROR) = predNAME  
		
		## create mean vector for 'mvrnorm'
		MU = c(COEF, predVAL)
		
		## create variance-covariance matrix for 'mvrnorm'
		## by putting error^2 in lower-right position of VCOV
		newVCOV = VCOV
		newVCOV[varPLACE, varPLACE] = predERROR^2
		## create MC simulation matrix
		simMAT = mvrnorm(n = nsim, mu = MU, Sigma = newVCOV, empirical = TRUE)
# 		print(summary(simMAT))
		## evaluate expression on rows of simMAT
		EVAL = try(eval(EXPR, envir = as.data.frame(simMAT)), silent = TRUE)
		if (inherits(EVAL, "try-error")) stop("There was an error evaluating the simulations!")
		
		## collect statistics
		PRED = data.frame(predVAL)
		colnames(PRED) = predNAME
		FITTED = predict(object, newdata = data.frame(PRED))
		MEAN.sim = mean(EVAL, na.rm = TRUE)
		SD.sim = sd(EVAL, na.rm = TRUE)
		MEDIAN.sim = median(EVAL, na.rm = TRUE)
		MAD.sim = mad(EVAL, na.rm = TRUE)
		QUANT = quantile(EVAL, c((1 - level)/2, level + (1 - level)/2))
		RES = c(FITTED, MEAN.sim, SD.sim, MEDIAN.sim, MAD.sim, QUANT[1], QUANT[2])
		outMAT = rbind(outMAT, RES)
	}
	  
	colnames(outMAT) = c("fit", "mean", "sd", "median", "mad", names(QUANT[1]), names(QUANT[2]))
	rownames(outMAT) = NULL
#     print(outMAT)
    return(outMAT)      
}

In [9]:
## Two-step nls2() model fit at a given level of precision (res)
fitMKmodel = function(alpha_trimmed, f_trimmed, res) {
	
	## First fitting using starting values (st)
# 	mod = tryCatch({
		
# 		## Starting values to fit the model  
# 		st = expand.grid(const_a=seq(-1,1,length.out=res + 1), const_b=seq(-1,1,length.out=res), const_c=seq(1,10,length.out=res + 1))
		
# 		## Fitting
# 		nls2::nls2(alpha_trimmed ~ const_a + const_b * exp(-const_c* f_trimmed), start=st, algorithm='brute-force', control=nls.control(maxiter=NROW(st)))
		
# 	}, error=function(cond) {}) ## Return condition of error when unable to fit
	
    st = expand.grid(const_a=seq(-1,1,length.out=res + 1), const_b=seq(-1,1,length.out=res), const_c=seq(1,10,length.out=res + 1))
    mod <- nls2::nls2(alpha_trimmed ~ const_a + const_b * exp(-const_c* f_trimmed),
                      start=st, algorithm='brute-force', control=nls.control(maxiter=NROW(st)))
    
	## If mod fails...
	if (length(mod) == 0) { return(NULL) }
	
	## Second fitting, starting from previous fit (mod)
	mod2 = tryCatch({
		nls2::nls2(alpha_trimmed ~ const_a + const_b * exp(-const_c* f_trimmed), start = mod, control=nls.control(maxiter=200))
		
	}, error=function(cond) {}) ## Same error handling than the previous step
	
	## If mod2 fails...
	if (length(mod2) == 0) { return(NULL) }
	
	## Return mod2 if fitted
	return(mod2)
}

In [10]:
approach <- 'aa'
test_mode <- 'aMKT'
pop <- 'EUR'
cutoff <- c(0.15)

data <- read.delim(paste(data_dir, "metaPops.tsv", sep=''))

gene_df <- read.csv(paste(data_dir, 'aa_genes.csv', sep=''), header=FALSE)

idx <- 1


gene_list <- as.character(gene_df[gene_df[,(gene_df[1,]=='W1') & (gene_df[2,]=='OFC')] == 1,1])
print(length(gene_list))
df <- data[data[,'ID'] %in% gene_list,]
df <- df[df[,'Population'] == pop,]
x <- makeSfs(df, cumu=TRUE)
daf <- x[['daf']]
div <- x[['div']]
# PopHumanAnalysisMod(genes=gene_list, pops=pop, test=test_mode, plot=FALSE, cutoff = cutoff)

[1] 14861


In [11]:
system.time({eMKT(daf,div)})

   user  system elapsed 
  0.366   0.000   0.395 

In [12]:
system.time({aMKT(daf,div)})

 [1] 53807.3036702  1072.9899274   489.4936429   146.2357019   -23.0912757
 [6]   -26.2452442   -86.4392706  -134.1159682  -179.4219284  -114.0639685
[11]  -144.1153842  -115.5236186    12.8039430    20.0596286    54.1894617
[16]    66.0994079    64.5667407    32.1820845     0.5553829    11.9272615


   user  system elapsed 
  0.626   0.220   0.582 

In [13]:
aMKT(daf,div)

 [1] 53807.3036702  1072.9899274   489.4936429   146.2357019   -23.0912757
 [6]   -26.2452442   -86.4392706  -134.1159682  -179.4219284  -114.0639685
[11]  -144.1153842  -115.5236186    12.8039430    20.0596286    54.1894617
[16]    66.0994079    64.5667407    32.1820845     0.5553829    11.9272615


model,a,b,c,alphaAsymptotic,ciLow,ciHigh,alphaOriginal
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
exponential,0.08762747,-2.478785,46.12182,0.08762747,0.07858098,0.09683765,-0.214113


In [14]:
if (nrow(daf) == 20) {
  		daf1 <- daf
  		daf1$daf10 <- sort(rep(seq(0.05,0.95,0.1),2)) ## Add column with the daf10 frequencies
        print(daf1$daf10)
  		daf1 <- daf1[c("daf10","Pi","P0")] ## Keep new frequencies, Pi and P0
  		daf1 <- aggregate(. ~ daf10, data=daf1, FUN=sum)	## Sum Pi and P0 two by two based on daf
  		colnames(daf1)<-c("daf","Pi","P0") ## Final daf columns name
  		
  		}

 [1] 0.05 0.05 0.15 0.15 0.25 0.25 0.35 0.35 0.45 0.45 0.55 0.55 0.65 0.65 0.75
[16] 0.75 0.85 0.85 0.95 0.95
